In [ ]:
%%bash
#pip install TextBlob
#conda install gensim

In [1]:
import pandas as pd
import numpy as np
import textblob as tb
import nltk
from nltk.corpus import stopwords as sw
from nltk.stem import *
import urllib2
from gensim import corpora, models, similarities

In [2]:
dropbox = "/Users/mr/Dropbox/moviemeta/"
movieplots = pd.read_csv(dropbox + 'movieplots.csv')
#this is a work around because CSV conversion has converted list into string
from ast import literal_eval
movieplots['plots'] = movieplots['plots'].apply(literal_eval)

In [3]:
def clean(row):
    try:
        row['year'] = int(row['year'])
        return row
    except:
        row['year'] = np.nan
        return row

In [29]:
movieplots = movieplots.apply(clean, axis = 1)
dftouse = movieplots[movieplots.year > 2014]

In [30]:
print movieplots.shape
print dftouse.shape

(259028, 5)
(16943, 5)


In [6]:
import findspark
findspark.init()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)   

In [31]:
plots_sdf = sqlsc.createDataFrame(dftouse)

In [32]:
def sentences(text):
    blob = tb.TextBlob(text)
    return blob.sentences

In [33]:
def stem(word):
    return PorterStemmer().stem(word)

In [43]:
def process(sents, dostop=True, dostem=True):
    stopwords = set(sw.words('english'))
    sents_processed  = []
    words_processed  = []
    for sent in sents:
        if dostop == True and dostem == True: 
            words = [stem(word.lower()) for word in sent.words if word not in stopwords]     
        elif dostop == True:
            words = [word.lower() for word in sent.words if word not in stopwords]
        elif dostem == True:
            words = [stem(word.lower()) for word in sent.words ]
        sents_processed.append(words)
        words_processed += words 
    return (sents_processed, words_processed)

In [44]:
plots = (plots_sdf[['title','plots']]
    .map(lambda x : (' '.join(x[1])))
)
docs = (plots
    .map(sentences)
    .map(process)          
).cache()
docs_sents = docs.map(lambda x : x[0]).collect()
docs_words = docs.map(lambda x : x[1]).collect()

In [45]:
dictionary = corpora.Dictionary(docs_words)
dictionary.save(dropbox +"testdict.dict") 

In [46]:
corpus = (dictionary.doc2bow(doc) for doc in docs_words)
corpora.MmCorpus.serialize(dropbox +"testcorpus.mm", corpus)

In [38]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]